In [3]:
%pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 883.4 kB/s eta 0:00:00m eta 0:00:010:00:02
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import date
import yfinance as yf
import pandas as pd
import duckdb
from dateutil.relativedelta import relativedelta

In [14]:
tickers_to_download = ['AAPL', 'GOOG','AMZN', 'MSFT', 'AMD', 'NVDA', 'IBM']

# Скачивание данных о котировках
def download_stock_data(ticker, start_date: date, end_date: date):
    data = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"), interval="1d", threads=True)
    # group_by='ticker'
    return data

def preload_date_for_ticker(db_connection, ticker: str, today: date) -> pd.DataFrame:
    start_date = today
    last_item = db_connection.sql("select stock_date from stock_data where ticker like '{}' order by stock_date desc limit 1".format(ticker))   
    if last_item.df().shape[0] == 0:
        start_date = today - relativedelta(years=1)
    else:
        start_date = last_item.df().iloc[0]['stock_date']
    
    print(start_date, "->", ticker)
    
    data = download_stock_data(ticker, start_date, today)

    for index, row in data.iterrows():
        insert_statement = """
            insert into stock_data(ticker, stock_date, open, close, high, low, adj_close, volume) values('{}', '{}', {}, {}, {}, {}, {}, {})  ON CONFLICT DO NOTHING;
        """.format(
                   ticker,
                   index,
                   row['Open'][ticker],
                   row['Close'][ticker],
                   row['High'][ticker],
                   row['Low'][ticker],
                   row['Adj Close'][ticker],
                   row['Volume'][ticker])
        # print(insert_statement)
        db_connection.sql(insert_statement)


def establish_db_connection():
    con = duckdb.connect("./stock_data.db")

    create_table_statement = """
        CREATE TABLE IF NOT EXISTS stock_data (
            ticker varchar(10) not null,
            stock_date datetime not null,
            open FLOAT not null,
            close FLOAT not null,
            high FLOAT not null,
            low FLOAT not null,
            adj_close FLOAT not null,
            volume FLOAT not null
        );
        
    """

    create_index_statement = """
        CREATE UNIQUE INDEX ticker_ts ON stock_data (ticker, stock_date);
    """

    con.sql(create_table_statement)

    try:
        con.sql(create_index_statement)
    except:
        print("index alredy exists")

    return con

In [11]:
con = establish_db_connection()

for t in tickers_to_download:
    preload_date_for_ticker(con, t,  date.today())

[*********************100%***********************]  1 of 1 completed

2024-01-20 -> AAPL



[*********************100%***********************]  1 of 1 completed

2024-01-20 -> GOOG



[*********************100%***********************]  1 of 1 completed

...

[*********************100%***********************]  1 of 1 completed


2024-01-20 -> NVDA


[*********************100%***********************]  1 of 1 completed


2024-01-20 -> IBM


[*********************100%***********************]  1 of 1 completed


In [12]:
last_item = con.sql("select stock_date from stock_data where ticker like '{}' order by stock_date desc limit 1".format("IBM"))   
last_item.df().iloc[0]['stock_date']
# start_date = last_item.df().iloc[0]['stock_data']


Timestamp('2025-01-17 00:00:00')

In [13]:
con.close()

In [22]:

def get_rid_of_outliers(source_df: pd.DataFrame): 
    print(source_df.columns)
    # Вычисление межквартильного размаха (IQR)
    Q1 = source_df['Close'].quantile(0.25)  # Первый квартиль (25-й процентиль)
    Q3 = source_df['Close'].quantile(0.75)  # Третий квартиль (75-й процентиль)
    IQR = Q3 - Q1                        # Межквартильный размах

    # Определение границ для выбросов
    lower_bound = Q1 - 1.5 * IQR  # Нижняя граница
    upper_bound = Q3 + 1.5 * IQR  # Верхняя граница

    # Пометка выбросов
    source_df['is_outlier'] = (source_df['Close'] < lower_bound) | (source_df['Close'] > upper_bound)
    window = 5  # Количество дней для среднего
    source_df['price_corrected'] = source_df['Close']
    source_df.loc[source_df['is_outlier'], 'Close'] =  source_df['price_corrected'].rolling(window=window, center=True).mean()
    source_df.drop(columns=['price_corrected'], inplace=True)

In [31]:
data = yf.download('AMZN', start='2024-01-20', end='2025-01-26', interval="1d", threads=True, group_by='Ticker', auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [14]:
data['AMZN'].columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object', name='Price')

In [32]:
df = data['AMZN']

In [33]:
get_rid_of_outliers(df)

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object', name='Price')


/var/folders/yd/h4jp1p2n15l6yrq6ndgwygxh0000gn/T/ipykernel_46793/1721976483.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_df['is_outlier'] = (source_df['Close'] < lower_bound) | (source_df['Close'] > upper_bound)
/var/folders/yd/h4jp1p2n15l6yrq6ndgwygxh0000gn/T/ipykernel_46793/1721976483.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_df['price_corrected'] = source_df['Close']
/var/folders/yd/h4jp1p2n15l6yrq6ndgwygxh0000gn/T/ipykernel_46793/1721976483.py:17: SettingWithCopyWarning:

In [34]:
# df.ffill(inplace=True)
df[df['is_outlier'] == True]

Price,Open,High,Low,Close,Volume,is_outlier
Date,,,,,,
2024-12-06,220.750000,227.149994,220.600006,223.373999,44178100,True
2024-12-09,227.210007,230.080002,225.669998,225.793997,46819400,True
2024-12-10,226.089996,229.059998,224.199997,227.477997,31199900,True
2024-12-11,226.410004,231.199997,226.259995,227.563998,35385800,True
2024-12-12,229.830002,231.089996,227.630005,228.931998,28204100,True
2024-12-13,228.399994,230.199997,225.860001,230.153998,28768100,True
2024-12-16,230.229996,233.000000,228.009995,228.206000,37552100,True
2024-12-17,232.389999,232.729996,227.850006,227.069998,35948100,True
2024-12-19,224.910004,226.089996,222.919998,224.987997,39918700,True
